In [ ]:
!pip install kaggle

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
!mkdir ~/.kaggle

In [ ]:
#! cp kaggle.json  ~/.kaggle/

In [ ]:
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/"""your kaggle json file"""

In [ ]:
! chmod 600 ~/.kaggle/"""your kaggle json file"""

In [ ]:
!kaggle datasets download pranavraikokte/pneumoniadataset

In [ ]:
! unzip pneumoniadataset.zip

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_selection import SelectKBest, chi2
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:
# Set up the data generators for loading and preprocessing the images
data_dir = '/content/Pnuemonia'
batch_size = 32
img_size = (224, 224)

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [ ]:
train_data = train_datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True)

In [ ]:
base_model = tf.keras.applications.VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(img_size[0], img_size[1], 3))

In [ ]:
train_features = base_model.predict(train_data, verbose=1)

In [ ]:
train_features = np.reshape(train_features, (train_features.shape[0], -1))

In [ ]:
svm = SVC(kernel='linear', C=1.0, random_state=42)
svm.fit(train_features, train_data.labels)


In [ ]:
train_preds = svm.predict(train_features)
print(classification_report(train_data.labels, train_preds))
print(confusion_matrix(train_data.labels, train_preds))

In [ ]:
selector = SelectKBest(chi2, k=1000)
train_features = selector.fit_transform(train_features, train_data.labels)

In [ ]:
svm = SVC(kernel='linear', C=1.0, random_state=42)
svm.fit(train_features, train_data.labels)


In [ ]:
train_preds = svm.predict(train_features)
print(classification_report(train_data.labels, train_preds))
print(confusion_matrix(train_data.labels, train_preds))


In [ ]:
image_path="/content/Pnuemonia/Positive/patient00085_study1_view1_frontal.jpg"

In [ ]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = tf.keras.applications.vgg16.preprocess_input(x)
    return x

In [ ]:


def extract_features(image_path):
    x = preprocess_image(image_path)
    features = base_model.predict(x)
    features = np.reshape(features, (features.shape[0], -1))
    return features
new_features=extract_features(image_path)
new_features=selector.transform(new_features)
prediction=svm.predict(new_features)

print(prediction)


In [ ]:
from joblib import dump, load
dump(svm, 'svm.joblib')
dump(selector, 'selector.joblib')


In [ ]:
loadedSvm=load('svm.joblib')
loadedSelector=load('selector.joblib')
# Load the image and preprocess it
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = tf.keras.applications.vgg16.preprocess_input(x)
    return x

def extract_features(image_path):
    x = preprocess_image(image_path)
    features = base_model.predict(x)
    features = np.reshape(features, (features.shape[0], -1))
    return features

new_features = extract_features(image_path)
new_features = loadedSelector.transform(new_features)
prediction = loadedSvm.predict(new_features)

print(prediction)

if prediction==0:
  print("The person has pneumonia")

else:
  print("The person is healthy")

In [ ]:
import seaborn as sns
sns.heatmap(train_features)

In [ ]:
# steps for implementing cnn
    #Choose a dataset and import libraries
    #Prepare a dataset for training ie assigning paths and creating categories(labels), resizing our images

    #Create training data
    # Shuffle the dataset
    #assign labels and features
    # Normalising X and converting labels into categorical data
    #Split X and Y for use in CNN
    #Define, compile and train the CNN Model
    # Accuracy and score of model
    # we are going to utilize the kaggle dataset that is made up of 1000 images 500 for pneumonia and 500 for normal chest x_rays

In [ ]:
#The libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
import numpy as np
# we use glob to retrieve files in folders and sub folder

In [ ]:
# import libraries
import tensorflow as tf
import os
import cv2
import imghdr
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy
from tensorflow.keras.models import load_model
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

In [ ]:
# To see available devices
gpus=tf.config.experimental.list_physical_devices('CPU')
gpus

In [ ]:
# limiting tensor from using all gpu space
gpus=tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
# loading images
# imghdr allows us to check extensions

data_dir='/content/Pnuemonia'

In [ ]:
# checking subfolders
os.listdir(data_dir)


In [ ]:
# checking images in negative folder
os.listdir(os.path.join(data_dir, 'Negative'))

In [ ]:
# accepted extensions
image_exts=['jpeg', 'jpg', 'bmp', 'png']


In [ ]:
# running extensions
image_exts

In [ ]:
# image class print negative or positive
for image_class in os.listdir(data_dir):
    print(image_class)

In [ ]:
# for loop 2 below prints every single image in that directory
for image_class in os.listdir(data_dir):
    for image in os.listdir(os.path.join(data_dir, image_class)):
        print(image)

In [ ]:
# checking if open cv allows us to open all images
img=cv2.imread(os.path.join('/content/Pnuemonia', 'Negative', 'patient00012_study3_view1_frontal.jpg'))


In [ ]:
# checking the properties of our image
img.shape

In [ ]:
# opening our image
plt.imshow(img)


In [ ]:
# changing the color to be normal
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
# to get rid of <matplotlib.image.AxesImage at 0x7f6fff0f1570>
plt.show()

In [ ]:
# removing corrupted images
for image_class in os.listdir(data_dir):
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path=os.path.join(data_dir, image_class, image)
        try:
            img=cv2.imread(image_path)
            tip=imghdr.what(image_path)
            if tip not in image_exts:
                print('Image not in extension list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e:
            print('Issue with image {}'.format(image_path))
            # os.remove(image_path)

In [ ]:
# to show what keras can do
tf.keras.utils.image_dataset_from_directory??

In [ ]:
# now loading our data
# adds classes and labels and some preprocessing
#building our data pipeline
data=tf.keras.utils.image_dataset_from_directory('/content/Pnuemonia')


In [ ]:
data

In [ ]:
# in order to check the data we first convert it into a numpy iterator first
#looping through our data pipeline
data_iterator=data.as_numpy_iterator()

In [ ]:
data_iterator

In [ ]:
# to run the .next line
#accessing our data pipeline
batch=data_iterator.next()

In [ ]:

# looking through our data pipeline
batch


In [ ]:
# looking at the shape of the batch
# images represented as numpy arrays
batch[0].shape

In [ ]:
#labels
batch[1]

In [ ]:
len(batch)
# it is two because the first part is the images and the second is the labels

In [ ]:
# to check which class ie either 0 or 1 is attached to which class ie Negative or positive
fig, ax=plt.subplots(ncols=4, figsize=(20, 20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

In [ ]:
# preprocessing the data
# scaling the data to be between 0 and 1 not between 0 and 255
# done when we prefetch data ie transformation in pipeline
data=data.map(lambda  x,y:(x/255, y))

In [ ]:
# taking a looking at our data batch, we can see that it is scaled in the pipeline
scaled_iterator=data.as_numpy_iterator()


In [ ]:
batch=scaled_iterator.next()

In [ ]:
batch[0].max()

In [ ]:
fig, ax=plt.subplots(ncols=4, figsize=(20, 20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img)
    ax[idx].title.set_text(batch[1][idx])

In [ ]:
# checking the length of data
len(data)

In [ ]:
# Splitting our data
train_size=int(len(data)*.7)
val_size=int(len(data)*.2)
test_size=int(len(data)*.1)

In [ ]:
# size of training data
train_size

In [ ]:
# size of validation data
val_size


In [ ]:
# size of test data
test_size

In [ ]:
# checking if all the length of the data adds up
# training for training, validation for testing while training and testing for testing after completion
train_size+val_size+test_size

In [ ]:
# in order to use this data we use the take and skip variable inside the tensorflow dataset pipeline
train=data.take(train_size)
val=data.skip(train_size).take(val_size)
test=data.skip(train_size+val_size).take(test_size)

In [ ]:
# Creating our deep learning model
# Build a deep learning model
model=Sequential()

In [ ]:
# adding our layers
# EG in the first convolution layer we have got 16 filters, which have 3 pixels by 3 pixels and we have a stride of 1
model.add(Conv2D(16, (3, 3), 1, activation='relu', input_shape=(256, 256, 3)))
model.add(MaxPooling2D())

model.add(Conv2D(32, (3, 3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(16, (3, 3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# compiling the model
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
# Training
# we create a log directory
logdir='/home/mylaptop/Downloads/archive (5)/Logs/'

In [ ]:
# useful if you need to save your model at a particular checkpoint or if you want to do a specific logging
tensorboard_callback=tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
# fitting our model
#epochs=how long we are going to train for ie one run over the training data
hist=model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])

In [ ]:
hist.history

In [ ]:
# plot perfomance
# visualizing loss
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
plt.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()
# check regularization

In [ ]:
# visualizing accuracy
fig=plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
#Evaluating perfomance
# establishing instances of precision, recall and binary accuracy
pre=Precision()
re=Recall()
acc=BinaryAccuracy()

In [ ]:
# testing the imports in our data
for batch in test.as_numpy_iterator():
    X, y=batch
    yhat=model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

In [ ]:
# printing the results of prediction, recall and accuracy
print(f'Precision:{pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

In [ ]:
# Testing phase
# pneumonic test
#NormalHealthy
#pneumoniaPositive
img=cv2.imread('/content/Pnuemonia/Negative/patient00063_study3_view1_frontal.jpg')
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
# editing the image to fit our details.
resize=tf.image.resize(img, (256, 256))
plt.imshow(cv2.cvtColor(resize.numpy(), cv2.COLOR_BGR2RGB).astype(int))
plt.show()

In [ ]:

# encapsulating the image since our neural network expects many images but we are only passing one. and then showing its shape
np.expand_dims(resize, 0).shape

In [ ]:
# PREDICTING IF IT IS PNEUMONIC OR NOT
yhat=model.predict(np.expand_dims(resize/255, 0))

In [ ]:
#prediction
yhat
#NB pneumonic is 1
#Normal is 0

In [ ]:
if yhat>0.5:
    print("The person has pneumonia")
else:
    print("The person is normal")

In [ ]:
#model_feature_extractor = Sequential(model.layers[:-1])
#train_features = model_feature_extractor.predict(train)
#val_features = model_feature_extractor.predict(val)
#test_features = model_feature_extractor.predict(test)

train_data = ImageDataGenerator().flow_from_directory('/content/Pnuemonia/', target_size=(256, 256), batch_size=batch_size, class_mode=None)
train_labels = train_data.classes


from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# create an LDA object
lda = LinearDiscriminantAnalysis()

# fit the LDA model on the training data and transform the data
train_features = lda.fit_transform(train_features, train_labels)

# define a new model
model_lda = Sequential([
    Dense(64, activation='relu', input_shape=(train_features.shape[1],)),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# compile the model
model_lda.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', Precision(), Recall()])

model_lda.summary()


In [ ]:
hist.history

In [ ]:
# plot perfomance
# visualizing loss
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
plt.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()
# check regularization

In [ ]:
# visualizing accuracy
fig=plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
#Evaluating perfomance
# establishing instances of precision, recall and binary accuracy
pre=Precision()
re=Recall()
acc=BinaryAccuracy()

In [ ]:
# testing the imports in our data
for batch in test.as_numpy_iterator():
    X, y=batch
    yhat=model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

In [ ]:
# printing the results of prediction, recall and accuracy
print(f'Precision:{pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

In [ ]:
# Testing phase
# pneumonic test
#NormalHealthy
#pneumoniaPositive
img=cv2.imread('/content/Pnuemonia/Negative/patient00063_study3_view1_frontal.jpg')
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
# editing the image to fit our details.
resize=tf.image.resize(img, (256, 256))
plt.imshow(cv2.cvtColor(resize.numpy(), cv2.COLOR_BGR2RGB).astype(int))
plt.show()

In [ ]:
# encapsulating the image since our neural network expects many images but we are only passing one. and then showing its shape
np.expand_dims(resize, 0).shape

In [ ]:
# PREDICTING IF IT IS PNEUMONIC OR NOT
yhat=model.predict(np.expand_dims(resize/255, 0))

In [ ]:
#prediction
yhat
#NB pneumonic is 1
#Normal is 0

In [ ]:
if yhat>0.5:
    print("The person has pneumonia")
else:
    print("The person is normal")

In [ ]:
# Save the model
# This will create a file name pneumoniaPredictionModel.h5 in the models folder
model.save(os.path.join('/content/Pnuemonia/models', 'pneumoniaPredictionModel.h5'))



In [ ]:
#This is for reloading the model
new_model=load_model(os.path.join('/content/Pnuemonia/models', 'pneumoniaPredictionModel.h5'))
# read the model
# passing data to the model and predicting
yhat_new=new_model.predict(np.expand_dims(resize/255, 0))
if yhat_new>0.5:
  print("The person has pneumonia")
else:
  print("The person is healthy")


In [ ]:
import matplotlib.pyplot as plt
labels=['Support Vector Machine', 'Convolutional Neural Network']
accuracy=[100, 92.31]
precision=[100, 90.91]
recall=[100, 93.98]

fig, ax=plt.subplots()

bar_width=0.15

x_accuracy=np.arange(len(labels))

x_recall=x_accuracy+bar_width
x_precision=x_accuracy+2*bar_width

accuracy_bars=ax.bar(x_accuracy-bar_width/2, accuracy, bar_width, label='ACCURACY')
recall_bars=ax.bar(x_recall-bar_width/2, accuracy, bar_width, label='RECALL')
precision_bars=ax.bar(x_precision-bar_width/2, accuracy, bar_width, label='PRECISION')

ax.legend()

ax.set_ylim([0, 120])

ax.set_xticks(x_precision-bar_width/2)
ax.set_xticklabels(labels)

ax.set_title("PERFOMANCE METRICS BEFORE FEATURE SELECTION")
ax.set_xlabel("ALGORITHM USED")
ax.set_ylabel("PERCENTAGE ACHIEVED")

ax.set_xlim([x_accuracy[0]-bar_width, x_precision[-1] + bar_width])

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
labels=['Positive SVM', 'Negative SVM', 'CNN']
accuracy=[97, 97, 90.10]
precision=[98, 96, 91.76]
recall=[96, 97, 86.87]

fig, ax=plt.subplots()

bar_width=0.15

x_accuracy=np.arange(len(labels))

x_recall=x_accuracy+bar_width
x_precision=x_accuracy+2*bar_width

accuracy_bars=ax.bar(x_accuracy-bar_width/2, accuracy, bar_width, label='ACCURACY')
recall_bars=ax.bar(x_recall-bar_width/2, accuracy, bar_width, label='RECALL')
precision_bars=ax.bar(x_precision-bar_width/2, accuracy, bar_width, label='PRECISION')

ax.legend()

ax.set_ylim([0, 120])

ax.set_xticks(x_precision-bar_width/2)
ax.set_xticklabels(labels)

ax.set_title("PERFOMANCE METRICS AFTER FEATURE SELECTION")
ax.set_xlabel("ALGORITHM USED")
ax.set_ylabel("PERCENTAGE ACHIEVED")

ax.set_xlim([x_accuracy[0]-bar_width, x_precision[-1] + bar_width])

plt.show()